In [1]:
library(topicmodels)

# Load data

In [6]:
data('AssociatedPress')
AssociatedPress

<<DocumentTermMatrix (documents: 2246, terms: 10473)>>
Non-/sparse entries: 302031/23220327
Sparsity           : 99%
Maximal term length: 18
Weighting          : term frequency (tf)

In [47]:
data_fpath <- '../av_survey_data/bikepgh_av_survey.csv' # fill the path to this file here
survey_data <- read.csv(data_fpath, na.strings=c(''))
survey_data

participant_id,age,av_disclose_performance,av_reduce_injuries,av_report_safety,av_school_zone,av_share_data,av_speed_limit,av_two_drivers,bikegph_member,⋯,other_comments,own_car,own_smartphone,paying_attention_av_news,positive_av_interaction,regulate_av_testing,start_date,thoughts_pgh_av_testing,year,zipcode
0,NA,NA,Yes,NA,No,Yes,No,NA,NA,⋯,NA,NA,NA,To a moderate extent,NA,Not sure,02/22/2017 10:11:47 AM PST,Approve,2017,15216
1,NA,NA,Yes,NA,Not sure,Yes,Yes,NA,NA,⋯,"I would really like them to share their data with City Planning so we aren't just being guinea pigs for no payoff, but otherwise I am pretty in favor of them.",NA,NA,To some extent,NA,Yes,02/22/2017 10:15:25 AM PST,Neutral,2017,15224
2,NA,NA,Yes,NA,No,Yes,No,NA,NA,⋯,NA,NA,NA,To a moderate extent,NA,Yes,02/22/2017 10:17:08 AM PST,Approve,2017,15206
3,NA,NA,Maybe,NA,Yes,Yes,Not sure,NA,NA,⋯,i have spent a lot of time conversing with people who work at uber and third parties about these technologies and the responsibility that's carried along with using our city as a testing ground. feel free to call me at 5129448987 anna bieberdorf,NA,NA,To a large extent,NA,Yes,02/22/2017 10:19:36 AM PST,Disapprove,2017,15201
4,NA,NA,Maybe,NA,Yes,Yes,Yes,NA,NA,⋯,NA,NA,NA,To a moderate extent,NA,Yes,02/22/2017 10:27:29 AM PST,Somewhat Approve,2017,15224
5,NA,NA,Yes,NA,Not sure,Yes,Yes,NA,NA,⋯,NA,NA,NA,To a moderate extent,NA,Yes,02/22/2017 10:29:01 AM PST,Approve,2017,15201-1720
6,NA,NA,Yes,NA,No,Yes,Yes,NA,NA,⋯,"I don't think that pedestrians and cyclists should fear this technology. However, we should be cautious of our personal safety around unmanned vehicles as we would with any vehicle!",NA,NA,To a large extent,NA,Yes,02/22/2017 10:30:07 AM PST,Approve,2017,15215
7,NA,NA,Yes,NA,Not sure,Yes,Not sure,NA,NA,⋯,"I think eventually they'll be safer for bicyclists and pedestrians to interact with than human-operated vehicles. I'm not sure we're at that point yet, though.",NA,NA,To a moderate extent,NA,Yes,02/22/2017 10:32:10 AM PST,Somewhat Approve,2017,15227
8,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,02/22/2017 10:37:00 AM PST,NA,2017,NA
9,NA,NA,Yes,NA,No,Yes,No,NA,NA,⋯,NA,NA,NA,To a large extent,NA,Yes,02/22/2017 10:38:05 AM PST,Approve,2017,15217


In [48]:
nrow(survey_data)

[1] 1608

# Choose one of the text fields

In [49]:
colname <- 'interaction_details'
colname

[1] "interaction_details"

In [52]:
# Get non-empty rows from that columns
filtered_data <- subset(survey_data, !is.na(survey_data[colname]))
nrow(filtered_data)

[1] 943

# Tokenize (split text into words)
This may seem trivial, but you'll want to detach punctuation from words, since "person" and "person," aren't very different. And what about contractions such as "I'm"? Will you want to lowercase everything or is there some distinction between "polish" and "Polish" you'd want to preserve?

You'll also want to think about "stopwords", function words such as "the" and "and", or "or" and "that". Counts for these words are often distracting to machine learning models, and they're often removed unless there may be important or meaningful variation in stopword usage.

# Extract features (words to numbers)
One of the simplest ways to get documents into numeric format for machine learning is to simply count each unique word and treat each document as collection of these counts. For example, "the dog barked loudly at the hat" would become {the: 2, dog: 1, barked: 1, loudly: 1, at: 1, hat: 1}. Each unique word in the vocabulary is usually given an ID. Because order information is lost, this is referred to as the "bag-of-words" model of documents.

In [ ]:
# Make term-document matrix